# DataAnalysis

## Paquetes

In [25]:
import pandas as pd
import numpy as np


import plotly.graph_objects as go

## Funciones

In [18]:
# Función para extraer coordenadas x e y
def extract_coordinates(point):
    point = point.strip("()")
    x, y = point.split(", ")
    return int(x), int(y)

## Código

In [21]:
# Read the data from the excel file
data = pd.read_excel('data/output_points_6.xlsx')

# Split columns into two
for column in data.columns:
    data[column + ' (x)'] = data[column].apply(lambda p: extract_coordinates(p)[0])
    data[column + ' (y)'] = data[column].apply(lambda p: extract_coordinates(p)[1])

# Drop the original columns
data = data.drop(columns=data.columns[:6])

# Take Left Point 2 and Right Point 2
data_point_2 = data[['Left Point 2 (x)', 'Left Point 2 (y)', 'Right Point 2 (x)', 'Right Point 2 (y)']]

# Delete rows with 0 values, reindex
data_point_2 = data_point_2[(data_point_2.T != 0).all()]
data_point_2 = data_point_2.reset_index(drop=True)

# Find first row with the value equal to -2
index = data_point_2.index[data_point_2['Left Point 2 (x)'] == -2].tolist()[0]

# Delete rows after the first row with the value equal to -2
data_point_2 = data_point_2.iloc[:index]

# Change -1 values to nan
data_point_2 = data_point_2.replace(-1, np.nan)

# Complete nan by interpolation
data_point_2 = data_point_2.interpolate()

# Delete column Right Point 2 (x)
data_point_2 = data_point_2.drop(columns='Right Point 2 (x)')

# Rename columns: x, y, z
data_point_2.columns = ['x', 'y', 'z']

# Show the data
print(data_point_2.head(20))


        x      y    z
0   412.0  315.0  370
1   377.0  258.0  295
2   342.0  201.0  221
3   300.0  160.0  163
4   273.0  131.0   82
5   257.0  111.0   73
6   265.0  103.0   45
7   269.0  110.0   50
8   267.5  111.0   67
9   266.0  112.0   75
10  268.0  109.0   87
11  263.0  101.0   95
12  263.0   99.0   95
13  260.0   94.0   99


In [36]:
df = data_point_2.copy()

# Crear los tamaños de los marcadores, haciendo el último punto más grande
marker_sizes = [5] * (len(df) - 1) + [10]  # Último punto con tamaño 10

# Crear el gráfico 3D interactivo
fig = go.Figure()

# Añadir la trayectoria con todos los puntos menos el último
fig.add_trace(go.Scatter3d(
    x=df['x'][:-1],
    y=df['y'][:-1],
    z=df['z'][:-1],
    mode='lines+markers',
    marker=dict(size=5),
    line=dict(width=2),
    name = 'Trayectoria'
))

# Añadir el último punto con un tamaño más grande
fig.add_trace(go.Scatter3d(
    x=[df['x'].iloc[-1]],
    y=[df['y'].iloc[-1]],
    z=[df['z'].iloc[-1]],
    mode='markers',
    marker=dict(size=10, color='red'),  # Color opcional para diferenciar el último punto
    name='Flor'
))

# Configurar el diseño del gráfico
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='Trayectoria 3D Interactiva'
)

# Configurar la cámara para que salga cuadrado y alejado de los puntos
camera = dict(
    eye=dict(x=1, y=1, z=1)
)

# Mostrar el gráfico
fig.show()